In [3]:
import os

In [4]:
%pwd

'c:\\Users\\user\\Downloads\\Text_Summarizer-main\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\user\\Downloads\\Text_Summarizer-main'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)  # Immutable configuration object
class DataTransformationConfig:
    root_dir: Path        # Directory to save transformed datasets
    data_path: Path       # Path to the input dataset
    tokenizer_name: str   # Tokenizer name as a string


In [8]:
import sys
import os

# Add src directory to sys.path
sys.path.append(os.path.join(os.getcwd(), 'src'))

from textSummariser.constants import *
from textSummariser.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        # Load the configuration and parameters from YAML files
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create necessary directories
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        # Check the structure of the loaded config
        print("Loaded config:", self.config)

        # Assuming self.config is a dictionary with a 'data_transformation' section
        config = self.config.get('data_transformation', None)
        
        if config is None or not all(key in config for key in ['root_dir', 'data_path', 'tokenizer_name']):
            raise KeyError("Missing required keys in 'data_transformation' configuration.")
        
        # Debug print to check config structure
        print("Data Transformation Config loaded:", config)
        
        create_directories([config['root_dir']])  # Ensure root_dir exists
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config['root_dir'],
            data_path=config['data_path'],
            tokenizer_name=config['tokenizer_name']
        )
        
        return data_transformation_config


In [16]:
import os
from textSummariser.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [17]:
'''class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    
    def convert_example_to_features(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings=self.tokenizer(example_batch['summary'],max_length=128,truncation=True)

        return{
      'input_ids':input_encodings['input_ids'],
      'attention_mask':input_encodings['attention_mask'],
      'labels':target_encodings['input_ids']
              }    
        
    def  convert(self):
         dataset_samsum = load_from_disk(self.config.datapath)
         dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched= True)
         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))'''

import os
from datasets import load_from_disk
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config):
        """
        Initialize the DataTransformation class with a configuration object.

        Args:
            config (DataTransformationConfig): Configuration object with:
                - root_dir: Path to the directory to save transformed datasets.
                - data_path: Path to the input dataset.
                - tokenizer_name: Name of the tokenizer.
        """
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)
    
    def convert_example_to_features(self, example_batch):
        """
        Convert a single example batch to features suitable for training.

        Args:
            example_batch (dict): A batch containing dialogue and summary.
        
        Returns:
            dict: Encoded input and target features.
        """
        input_encodings = self.tokenizer(
            example_batch['dialogue'], 
            max_length=1024, 
            truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'], 
                max_length=128, 
                truncation=True
            )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        """
        Load the dataset, process it, and save the transformed dataset.

        Raises:
            FileNotFoundError: If the dataset path does not exist.
        """
        # Validate dataset path
        if not os.path.exists(self.config.data_path):
            raise FileNotFoundError(f"Data path '{self.config.data_path}' does not exist.")
        
        # Load dataset
        dataset_samsum = load_from_disk(self.config.data_path)
        
        # Transform the dataset
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features, batched=True)
        
        # Ensure the root directory exists
        os.makedirs(self.config.root_dir, exist_ok=True)

        # Save the transformed dataset
        output_path = os.path.join(self.config.root_dir, "samsum_dataset")
        dataset_samsum_pt.save_to_disk(output_path)
         
         
            

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e    

[2024-12-19 07:10:19,097: INFO: common: created directory at: artifacts]
Loaded config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_root': 'artifacts/data_ingestion', 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}}
Data Transformation Config loaded: {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}
[2024-12-19 07:10:19,119: INFO: common: created dire

c:\Users\user\.conda\envs\textS\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\user\.conda\envs\textS\lib\site-packages\transf